<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2022, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-02-16 15:03:02  
@LastEditors: Liusen  
@LastEditTime: 2020-02-16 15:50:02   

# 定义模块和窗口

In [1]:
from IPython.display import display
import ipywidgets.widgets as widgets
import cv2
import time

# 线程功能操作库
import threading
import inspect
import ctypes
import matplotlib.pyplot as plt

In [2]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [4]:
import YB_Pcb_Car  #导入亚博智能专用的底层库文件

car = YB_Pcb_Car.YB_Pcb_Car()
car.Ctrl_Servo(1,93)
car.Ctrl_Servo(2,160)

ModuleNotFoundError: No module named 'YB_Pcb_Car'

# 车体状态显示部分进程

##### 自动驾驶

In [5]:
#转向PID输出值
TurnZ_PID_slider = widgets.FloatSlider(description='TurnZ_PID', min=-100, max=100.0, step=0.01, orientation='Vertical')
# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([TurnZ_PID_slider])
# display the container in this cell's output
display(slider_container)

In [6]:
#bgr8转jpeg格式
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [7]:
#摄像头组件显示
import cv2
import ipywidgets.widgets as widgets
import threading
import time
import numpy as np
import matplotlib.pyplot as plt
import PID

image_widget = widgets.Image(format='jpg', width=320, height=240)
display(image_widget)

ModuleNotFoundError: No module named 'PID'

In [8]:
#俯视图
image_widget_1 = widgets.Image(format='jpg', width=320, height=240)
display(image_widget_1)

Image(value=b'', format='jpg', height='240', width='320')

In [9]:
#归一化图+左线（淡绿色） 图像中线（紫色）实际中线（蓝色） 右线（绿色）
image_widget_2 = widgets.Image(format='jpg', width=320, height=240)
display(image_widget_2)

Image(value=b'', format='jpg', height='240', width='320')

In [10]:
image = cv2.VideoCapture(0)
image.set(3,640)
image.set(4,480)
image.set(5, 30)  #设置帧率
# fourcc = cv2.VideoWriter_fourcc(*"MPEG")
image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
image.set(cv2.CAP_PROP_BRIGHTNESS, 60) #设置亮度 -64 - 64  0.0  40
#image.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64  2.0  50
#image.set(cv2.CAP_PROP_EXPOSURE, 156) #设置曝光值 1.0 - 5000  156.0 156
#ret, frame = image.read()
#image_widget.value = bgr8_to_jpeg(frame)

True

# 以下正式开始自动驾驶的进程

In [11]:
global Z_axis_pid
Z_axis_pid = PID.PositionalPID(0.5, 0, 1)  #1.2 0 0.1   （0.8 0.0 1.0可以跑有点震荡 0.85 0 2.0）
global prev_left
prev_left = 0
global prev_right
prev_right = 0

NameError: name 'PID' is not defined

In [12]:
def Camera_display():
    global peaks_count
    global prev_left, prev_right
    t_start = time.time()
    fps = 0
    global Z_axis_pid
    
    while 1:
        #1、获取图像
        ret, frame = image.read()
        
        #帧率显示
        fps = fps + 1
        mfps = fps / (time.time() - t_start)
        cv2.putText(frame, "FPS: " + str(int(mfps)), (80,80), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (0,255,255), 8)
        
        #图像resize
        frame = cv2.resize(frame,(320,240))
        
        
        #2、透视变换 (查看俯视图)
        matSrc = np.float32([[0, 149],  [320, 149], [281, 72], [43, 72]])
        matDst = np.float32([[0,240], [320,240], [320,0], [0,0]])
        matAffine = cv2.getPerspectiveTransform(matSrc,matDst)# mat 1 src 2 dst
        dst = cv2.warpPerspective(frame,matAffine,(320,240))
        
        #画蓝色的矩形框，采集数据的时候采集的就是当前蓝色矩形框中的数据
        #src 4->dst 4 (左下角 右下角 右上角 左上角 )
        pts = np.array([[0, 149],  [320, 149], [281, 72], [43, 72]], np.int32)
        # 顶点个数：4，矩阵变成4*1*2维
        # OpenCV中需要将多边形的顶点坐标变成顶点数×1×2维的矩阵
        # 这里 reshape 的第一个参数为-1, 表示“任意”，意思是这一维的值是根据后面的维度的计算出来的       
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(frame, [pts],True, (255, 0, 0), 3) #True表示闭合，（255,0,0）是蓝色，3是线宽
        hsv = cv2.cvtColor(dst, cv2.COLOR_BGR2HSV)
        lower_red = np.array([0, 70, 72])
        upper_red = np.array([7, 255, 255])
        dst = cv2.inRange(hsv, lower_red, upper_red)
        dst=cv2.bitwise_not(dst)
        #cv.imwrite('./photos/red_mask.jpg', red_mask)
        #img_show(dst,"red")
        #3、转化图像二值化(看二值化之后的黑白图像）
        #dst_gray = cv2.cvtColor(dst, cv2.COLOR_RGB2GRAY)   #转灰度
        #dst_retval, dst_binaryzation = cv2.threshold(dst_gray, 120, 255, cv2.THRESH_BINARY)   #二值化
        dst_binaryzation = cv2.erode(dst, None, iterations=1)                    #腐蚀化操作
        
 
        # 当axis为0时,是压缩行,即将每一列的元素相加,将矩阵压缩为一行
        # 当axis为1时,是压缩列,即将每一行的元素相加,将矩阵压缩为一列(这里的一列是为了方便理解说的，实际上，在控制台的输出中，仍然是以一行的形式输出的)
        histogram = np.sum(dst_binaryzation[dst_binaryzation.shape[0]//2:, :], axis=0)  #我们这里是一维数组
        midpoint = np.int(histogram.shape[0]/2) #直方图垂直尺寸
        
        left_sum = np.sum(histogram[:20], axis=0)  #计算左边像素点之和
        right_sum = np.sum(histogram[300:], axis=0)  #计算右边像素点之和
        
        #print("left_sum =%d "%left_sum)
        #print("right_sum = %d"%right_sum)
        
        
        rightpoint = 320
        center_r = 159
        #print (histogram)
        #print(histogram[::-1])
        #plt.plot(histogram)
        #plt.plot(histogram[::-1])
        #plt.show()
        
        #4、获取黑线左右边线并绘制线条
        #获取左右线的位置
        leftx_base = np.argmin(histogram[:rightpoint], axis = 0)
        rightx_base = np.argmin(histogram[::-1][:rightpoint], axis = 0) #反转直方图取最右侧的值
        rightx_base = 319 - rightx_base

        dst_binaryzation = cv2.cvtColor(dst_binaryzation,cv2.COLOR_GRAY2RGB)
        cv2.line(dst_binaryzation,(159,0),(159,240),(255,0,255),2)  #图像中线 紫色 # fix liusen 200321  149->159
        lane_center = int((leftx_base + rightx_base)/2)  #左右线中间位置
        #print("lane_center")
        #print(lane_center)
        cv2.line(dst_binaryzation,(leftx_base,0),(leftx_base,240),(0,255,0),2)   # 赛道左侧位置
        cv2.line(dst_binaryzation,(rightx_base,0),(rightx_base,240),(0,255,0),2) # 赛道右侧位置
        cv2.line(dst_binaryzation,(lane_center,0),(lane_center,240),(255,0,0),2) # 实际中线位置
        
        left_sum_value = int(np.sum(histogram[:center_r], axis = 0))/159
        right_sum_value = int(np.sum(histogram[center_r:], axis = 0))/159
        #print("left_sum_value = %d", left_sum_value)
        #print("right_sum_value = %d", right_sum_value)
        #5、计算偏差值
        Bias = 159 - lane_center
        cv2.putText(dst_binaryzation, "FPS:  " + str(int(mfps)), (10,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        cv2.putText(dst_binaryzation, "Bias: " + str(int(Bias)), (10,35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        #print(Bias)
        
        #转向角PID调节
        Z_axis_pid.SystemOutput = Bias
        Z_axis_pid.SetStepSignal(0)
        Z_axis_pid.SetInertiaTime(0.5, 0.2)
        
        
        if Z_axis_pid.SystemOutput > 25: # 20
            Z_axis_pid.SystemOutput = 25
        elif Z_axis_pid.SystemOutput < -25:
            Z_axis_pid.SystemOutput = -25
            
        TurnZ_PID_slider.value = int(Z_axis_pid.SystemOutput)
        
       
        if leftx_base == 0 and rightx_base == 319:
            if prev_left > prev_right:
                car.Control_Car(-70, 60)
            elif prev_left < prev_right:
                car.Control_Car(70, -70)
                
            prev_left = 0
            prev_right = 0
            
        else:
            if Bias > 3:   #黑线在中线的左边，小车左转（左边速度减小，右边速度增大）
                #prev_left = 1
                #prev_right = 0
                if Bias > 140: 
                    car.Control_Car(-70, 60)
                    prev_left = 0
                    prev_right = 0
                else:
                    car.Control_Car(45+int(Z_axis_pid.SystemOutput), 45-int(Z_axis_pid.SystemOutput))
                time.sleep(0.001) 
            elif Bias < -3:    #黑线在中线的右边，小车右转   （左边速度增大，右边速度减小）
                #prev_right = 1
                #prev_left = 0
                if Bias < -140:   
                    car.Control_Car(60, -70)
                    prev_left = 0
                    prev_right = 0
                else:
                    car.Control_Car(45+int(Z_axis_pid.SystemOutput), 45-int(Z_axis_pid.SystemOutput))
                time.sleep(0.001)

            else:
                car.Car_Run(45, 45)
     
        


        if left_sum != right_sum:
            if left_sum < right_sum:
                prev_left = prev_left + 1
            elif right_sum < left_sum:
                prev_right = prev_right + 1

        

        image_widget.value = bgr8_to_jpeg(frame)
        image_widget_1.value = bgr8_to_jpeg(dst)
        image_widget_2.value = bgr8_to_jpeg(dst_binaryzation)



In [16]:
#自动驾驶进程启动
thread3 = threading.Thread(target=Camera_display)
thread3.setDaemon(True)
thread3.start()

C:\Users\HI\AppData\Local\Temp\ipykernel_3172\2806755357.py:3: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  thread3.setDaemon(True)


In [13]:
#结束自动驾驶进程
stop_thread(thread3)
car.Car_Stop()

NameError: name 'thread3' is not defined

In [14]:
image.release()